# Ray Tracing with Physgen Dataset

An example of using the ips framework (img_phy_sim).

The `ips.ray_tracing` part contains all useful functions to trace rays on 2D images.<br>
The `ips.img` provides very useful functions to work with images. Most of it are for showing images.

Contents:
- [Imports](#imports)
- [Data Loading](#data-loading)
- [Showing the Image](#showing-the-image)
- [Wall Maps](#wall-maps)
- [Ray Tracing](#ray-tracing)
- [Draw Rays](#draw-rays)
- [Comparison with Physgen Reflections](#comparison-with-physgen-reflections)

---
### Imports

In [ ]:
import sys
sys.path += [".."] # os.path.abspath("../img-phy-sim")

import img_phy_sim as ips
from physgen_dataset import PhysGenDataset, get_image  # local module

import os
import random

import numpy as np
from torch.utils.data import DataLoader

---
### Data Loading

If already downloaded:

In [ ]:
data_path = "../datasets/physgen_train_raw/osm/"

img_src = data_path + random.sample(os.listdir(data_path), 1)[0]
should_scale = True
img_src

Or:

In [ ]:
# img_src = "./cache_img.png"
# ips.img.save(
#     get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="complex_only", 
#               shuffle=True, return_output=False, as_numpy_array=True), 
#     img_src, should_scale=False)
# should_scale = False

# img_src

---
### Showing the Image

In [ ]:
img = ips.img.open(src=img_src, should_scale=should_scale, should_print=True)
ips.img.imshow(img, size=4, axis_off=False)

In [ ]:
ips.img.show_image_with_line_and_profile(imgs=[img], axis='row', index=None, titles=None, figsize=(10, 8));

---
### Wall Map

The Wall Map shows the angle of the walls everything else is infinite.

This function will be automatically called from the `ips.ray_tracing.trace_beams` function. Therefore most of the time this function does not get called alone.

It features:
- `thickness`: custom thickness of the wall-map
- `wall_values`: auto edge detection or edge detection by given pixel-values, so possible are these 3 methods: None or 240 or [1.0, 0.5, 0.4]

In [ ]:
wall_map = ips.ray_tracing.get_wall_map(img, wall_values=None, thickness=0)
ips.img.imshow(wall_map, size=4, axis_off=False)

In [ ]:
ips.img.show_image_with_line_and_profile(imgs=[wall_map], axis='row', index=None, titles=None, figsize=(10, 8));

In [ ]:
ips.img.show_image_with_line_and_profile(imgs=[wall_map], axis='column', index=None, titles=None, figsize=(10, 8));

In [ ]:
ips.img.imshow(ips.ray_tracing.get_wall_map(img, wall_values=None, thickness=1), size=4, axis_off=False)

In [ ]:
ips.img.imshow(ips.ray_tracing.get_wall_map(img, wall_values=None, thickness=2), size=4, axis_off=False)

In [ ]:
ips.img.imshow(ips.ray_tracing.get_wall_map(img, wall_values=0.0, thickness=0), size=4, axis_off=False)

In [ ]:
ips.img.plot_image_with_values(wall_map, block_size=8, 
                               cmap='gray', title=None, 
                               font_size=10, save_to=None)

---
### Ray Tracing

The `ips.ray_tracing.trace_beams` is the function to use for simulating the 2D ray tracing on an image.

Features are:
- Setting custom startposition for raytracing
- Adding custom beam shooting positions in degree (where 0° is the east/right of the image and 90° is south/bottom and so on)
- Setting reflexion order (how maxium reflexions should be calculated)
- Setting if the border of the image should be reflective or not
- And setting if the input image should be scaled and if the rays itself should be scaled
- You can also set the "wall" object values, which should get detected as objects with collision. If set to None, the programm will find all clear edges.

> Notice that the "wall" values will changed if you scale the image

Use the `ips.ray_tracing.print_rays_info` function to print informations about your rays.

In [ ]:
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=[0, 45],
                                   wall_values=None, 
                                   wall_thickness=1,
                                   img_border_also_collide=False,
                                   reflexion_order=0,
                                   should_scale_rays=True,
                                   should_scale_img=True)
ips.ray_tracing.print_rays_info(rays)

In [ ]:
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=10),
                                   wall_values=None, 
                                   wall_thickness=1,
                                   img_border_also_collide=False,
                                   reflexion_order=1,
                                   should_scale_rays=True,
                                   should_scale_img=True)
ips.ray_tracing.print_rays_info(rays)

Scaling (notice that both images are equal)

In [ ]:
rays_ = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=[22, 56, 90, 146, 234, 285, 320],
                                   wall_values=0.0, 
                                   wall_thickness=0,
                                   img_border_also_collide=False,
                                   reflexion_order=2,
                                   should_scale_rays=False,
                                   should_scale_img=True)
print("\nNot Scaled Ray:", rays_[0][0][:min(len(rays_[0][0])-1, 3)])

ray_img_1 = ips.ray_tracing.draw_rays(rays_, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=False, original_max_width=None, original_max_height=None)

rays_ = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=[22, 56, 90, 146, 234, 285, 320],
                                   wall_values=0.0, 
                                   wall_thickness=0,
                                   img_border_also_collide=False,
                                   reflexion_order=2,
                                   should_scale_rays=True,
                                   should_scale_img=True)

print("Scaled Ray:", rays_[0][0][:min(len(rays_[0][0])-1, 3)])

ray_img_2 = ips.ray_tracing.draw_rays(rays_, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.advanced_imshow([ray_img_1, ray_img_2], title=None, image_width=8, axis=False,
                        color_space="gray", cmap=None, cols=2, save_to=None,
                        hspace=0.2, wspace=0.2,
                        use_original_style=False, invert=False)

Image Border Collision + Reflexion Order Showcase

In [ ]:
rays_ = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=[22, 56, 90, 146, 234, 285, 320],
                                   wall_values=0.0, 
                                   wall_thickness=0,
                                   img_border_also_collide=True,
                                   reflexion_order=2,
                                   should_scale_rays=False,
                                   should_scale_img=True)

ray_img = ips.ray_tracing.draw_rays(rays_, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=False, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

In [ ]:
rays_ = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=[42],
                                   wall_values=0.0, 
                                   wall_thickness=0,
                                   img_border_also_collide=True,
                                   reflexion_order=20,
                                   should_scale_rays=False,
                                   should_scale_img=True)

ray_img = ips.ray_tracing.draw_rays(rays_, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=False, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

Change position

In [ ]:
rays_ = ips.ray_tracing.trace_beams(rel_position=[0.9, 0.9], 
                                   img_src=img_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=10),
                                   wall_values=0.0, 
                                   wall_thickness=0,
                                   img_border_also_collide=False,
                                   reflexion_order=3,
                                   should_scale_rays=True,
                                   should_scale_img=True)

ray_img = ips.ray_tracing.draw_rays(rays_, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

Loading and Saving

In [ ]:
rays_saved = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                        img_src=img_src, 
                                        directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=10),
                                        wall_values=None, 
                                        wall_thickness=1,
                                        img_border_also_collide=False,
                                        reflexion_order=1,
                                        should_scale_rays=True,
                                        should_scale_img=True)
ips.ray_tracing.print_rays_info(rays_saved)
ips.ray_tracing.save(path="./my_awesome_rays.txt", rays=rays_saved)

rays_loaded = ips.ray_tracing.open(path="./my_awesome_rays.txt")

print(f"\n{rays_saved == rays_loaded=}\n")
ips.ray_tracing.print_rays_info(rays_loaded)

Merging rays

In [ ]:
rays_1 = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=60),
                                   wall_values=None, 
                                   wall_thickness=1,
                                   img_border_also_collide=False,
                                   reflexion_order=1,
                                   should_scale_rays=True,
                                   should_scale_img=True)
ips.ray_tracing.print_rays_info(rays_1)

rays_2 = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=90, offset=3),
                                   wall_values=None, 
                                   wall_thickness=1,
                                   img_border_also_collide=False,
                                   reflexion_order=1,
                                   should_scale_rays=True,
                                   should_scale_img=True)
ips.ray_tracing.print_rays_info(rays_2)

merged_rays = ips.ray_tracing.merge(rays_1, rays_2)
ips.ray_tracing.print_rays_info(merged_rays)

del merged_rays, rays_1, rays_2

Let's beeing fancy and try out the merging furthermore 

In [ ]:
rays_ = None
max_ = 5
for x in range(0, max_):
    for y in range(0, max_):
        new_rays = ips.ray_tracing.trace_beams(rel_position=[x/(max_-1), y/(max_-1)], 
                                                img_src=img_src, 
                                                directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=45),
                                                wall_values=0.0, 
                                                wall_thickness=0,
                                                img_border_also_collide=False,
                                                reflexion_order=2,
                                                should_scale_rays=True,
                                                should_scale_img=True)
        rays_ = ips.ray_tracing.merge(rays_, new_rays) if rays_ else new_rays

ray_img = ips.ray_tracing.draw_rays(rays_, 
                                    detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=0.5, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="random")
del rays_

In [ ]:
rays_ = None
max_ = 15
for x in range(0, max_):
    for y in range(0, max_):
        new_rays = ips.ray_tracing.trace_beams(rel_position=[x/(max_-1), y/(max_-1)], 
                                                img_src=img_src, 
                                                directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=45),
                                                wall_values=0.0, 
                                                wall_thickness=0,
                                                img_border_also_collide=False,
                                                reflexion_order=2,
                                                should_scale_rays=True,
                                                should_scale_img=True)
        rays_ = ips.ray_tracing.merge(rays_, new_rays) if rays_ else new_rays

ray_img = ips.ray_tracing.draw_rays(rays_, 
                                    detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=0.5, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="random")
del rays_

In [ ]:
rays_ = None
max_ = 3
for x in range(0, max_):
    for y in range(0, max_):
        new_rays = ips.ray_tracing.trace_beams(rel_position=[x/(max_-1), y/(max_-1)], 
                                                img_src=img_src, 
                                                directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=10),
                                                wall_values=0.0, 
                                                wall_thickness=0,
                                                img_border_also_collide=False,
                                                reflexion_order=2,
                                                should_scale_rays=True,
                                                should_scale_img=True)
        rays_ = ips.ray_tracing.merge(rays_, new_rays) if rays_ else new_rays

ray_img = ips.ray_tracing.draw_rays(rays_, 
                                    detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=0.5, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="random")
del rays_

Let's make it even more fancy and add collision to the rays

In [ ]:
rays_ = None
ray_img = None
max_ = 5
offset = 0
for x in range(0, max_):
    for y in range(0, max_):
        new_rays = ips.ray_tracing.trace_beams(rel_position=[x/(max_-1), y/(max_-1)], 
                                                img_src=ray_img if type(ray_img) != type(None) else img_src, 
                                                directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=45, offset=offset),
                                                wall_values=[0.0, 0.5], 
                                                wall_thickness=0,
                                                img_border_also_collide=False,
                                                reflexion_order=2,
                                                should_scale_rays=True,
                                                should_scale_img=True)
        offset += np.random.random() *90
        # rays_ = ips.ray_tracing.merge(rays_, new_rays) if rays_ else new_rays

        ray_img = ips.ray_tracing.draw_rays(new_rays, 
                                            detail_draw=False, 
                                            output_format="single_image", 
                                            img_background=ray_img if type(ray_img) != type(None) else img, ray_value=0.5, ray_thickness=1, 
                                            img_shape=(256, 256), dtype=float, standard_value=0,
                                            should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="random")
del rays_

---
### Draw Rays

Export Rays into images. Here we cover some of the major features.

- Custom value of ray-traces
- Thickness of ray-traces
- Drawing on empty image or an existing image
    - Given image-shape, dtype and fill-value (standard-value)
- Scaling rays to the given image
- Different Format Types
    - One Image
    - Multiple Images (each ray on one image)
    - One Image and each channel is one ray
- Showing only the reflexions
- Give different values for different reflexion orders

**Basics**: Use the `ips.ray_tracing.draw_rays` to draw rays or use the `ips.ray_tracing.trace_and_draw_rays` to calculate the rays and draw/export these as an image and this for multiple images. 

Via the `ray_value` you set the pixel value which the rays should get in the image. The `ray_thickness` defines how thick the ray lines/points should get plottet /drawn on the image.

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=2, ray_thickness=4, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=24, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.show_image_with_line_and_profile(imgs=[ray_img], axis='column', index=None, titles=None, figsize=(11, 8));

**Detail Levels**: There are 2 levels of detail. 
1. One is exactly the way the rays got calculated pixel by pixel
2. The other is an approximation where lines are drawn from only he star tand the end point of each beam

See that here:

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(ray_img.shape)
ips.img.imshow(ray_img, size=5)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(ray_img.shape)
ips.img.imshow(ray_img, size=5)

**Background Image**: For each format you can choose if you want an empty image or an background image. For the empty image you can use the `image_shape`, `dtype` and the `standard_value` to set the base settings for your image.

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=0, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=255,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(ray_img.shape)
ips.img.imshow(ray_img, size=5)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(ray_img.shape)
ips.img.imshow(ray_img, size=5)

This comes also useful with the auto scaling with the image:

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=0, ray_thickness=1, 
                                    img_shape=(512, 512), dtype=float, standard_value=255,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(ray_img.shape)
ips.img.imshow(ray_img, size=4, axis_off=False)

**Formats:**

There are 3 formats: as a single image, every ray as a single image (so multiple image output) or one image output but with multiple channels and each ray is in one ray. <br>
In every format you can set a background image or a compelte new image.

<br><br>

We already saw the single image, let's have a look on the other 2 formats:

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True,
                                    output_format="multiple_images", 
                                    img_background=None, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(len(ray_img))
print(ray_img[0].shape)
ips.img.advanced_imshow(random.sample(ray_img, min(len(ray_img), 5)), title=None, image_width=5, axis=False,
                        color_space="RGB", cmap=None, cols=min(len(ray_img), 5), save_to=None,
                        hspace=0.2, wspace=0.2,
                        use_original_style=False, invert=False)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True,
                                    output_format="multiple_images", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(len(ray_img))
print(ray_img[0].shape)
ips.img.advanced_imshow(random.sample(ray_img, min(len(ray_img), 5)), title=None, image_width=10, axis=False,
                        color_space="RGB", cmap=None, cols=min(len(ray_img), 5), save_to=None,
                        hspace=0.2, wspace=0.2,
                        use_original_style=False, invert=False)

Let's see the channels and notice the output of the shape!

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True,
                                    output_format="channels", 
                                    img_background=None, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(ray_img.shape)
ips.img.imshow(ray_img[..., 0], size=5)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=True,
                                    output_format="channels", 
                                    img_background=img, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
print(ray_img.shape)
ips.img.imshow(ray_img[..., 0], size=5)

We can also pass multiple values for the rays which will give the different reflexion order different values 

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=[1.0, 2.0], ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="inferno")
ips.img.show_image_with_line_and_profile(imgs=[ray_img], axis='column', index=None, titles=None, figsize=(11, 8));

In [ ]:
rays_ = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=img_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=10),
                                   wall_values=None, 
                                   wall_thickness=1,
                                   img_border_also_collide=False,
                                   reflexion_order=5,
                                   should_scale_rays=True,
                                   should_scale_img=True)

ray_img = ips.ray_tracing.draw_rays(rays_, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=[1.0, 2.0, 3.0], ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="inferno")
ips.img.show_image_with_line_and_profile(imgs=[ray_img], axis='column', index=None, titles=None, figsize=(11, 8));

ray_img = ips.ray_tracing.draw_rays(rays_, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=None, ray_value=[1.0, 2.0, 3.0, 4.0, 5.0], ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="inferno")
ips.img.show_image_with_line_and_profile(imgs=[ray_img], axis='column', index=None, titles=None, figsize=(11, 8));

---
### Comparison with Physgen Reflections

In [ ]:
input_, output_ = get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="complex_only", 
               shuffle=False, return_output=True, as_numpy_array=True)

input_src = "./cache_img_1_input.png"
ips.img.save(input_, input_src, should_scale=False)

output_src = "./cache_img_1_output.png"
ips.img.save(output_, output_src, should_scale=False)

In [ ]:
ips.img.imshow(output_, size=5)

In [ ]:
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=input_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=10),
                                   wall_values=None, 
                                   wall_thickness=0,
                                   img_border_also_collide=False,
                                   reflexion_order=3,
                                   should_scale_rays=True,
                                   should_scale_img=False)
ips.ray_tracing.print_rays_info(rays)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=input_, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=output_, ray_value=0.5, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

In [ ]:
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=output_, ray_value=0.5, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None,
                                    show_only_reflections=True)
ips.img.imshow(ray_img, size=4)

On normal target (not only complex)

In [ ]:
# get image
input_, output_ = get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="standard", 
               shuffle=False, return_output=True, as_numpy_array=True)

input_src = "./cache_img_1_input.png"
ips.img.save(input_, input_src, should_scale=False)

output_src = "./cache_img_1_output.png"
ips.img.save(output_, output_src, should_scale=False)

ips.img.imshow(output_, size=5)

# calc rays
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=input_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=10),
                                   wall_values=None, 
                                   wall_thickness=0,
                                   img_border_also_collide=False,
                                   reflexion_order=3,
                                   should_scale_rays=True,
                                   should_scale_img=False)

# show rays on input
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=input_, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4, cmap="inferno")

# show rays on output
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=output_, ray_value=1.3, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

With more rays

In [ ]:
# get image
input_, output_ = get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="standard", 
               shuffle=False, return_output=True, as_numpy_array=True)

input_src = "./cache_img_1_input.png"
ips.img.save(input_, input_src, should_scale=False)

output_src = "./cache_img_1_output.png"
ips.img.save(output_, output_src, should_scale=False)

ips.img.imshow(output_, size=5)

# calc rays
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=input_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=1),
                                   wall_values=None, 
                                   wall_thickness=1,
                                   img_border_also_collide=False,
                                   reflexion_order=3,
                                   should_scale_rays=True,
                                   should_scale_img=False)

# show rays on input
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=input_, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

# show rays on output
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=output_, ray_value=1.3, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None)
ips.img.imshow(ray_img, size=4)

In [ ]:
# get image
input_, output_ = get_image(mode='train', variation="sound_reflection", input_type="osm", output_type="standard", 
               shuffle=False, return_output=True, as_numpy_array=True)

input_src = "./cache_img_1_input.png"
ips.img.save(input_, input_src, should_scale=False)

output_src = "./cache_img_1_output.png"
ips.img.save(output_, output_src, should_scale=False)

In [ ]:
# get image
ips.img.imshow(output_, size=5)

# calc rays
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=input_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=1),
                                   wall_values=None, 
                                   wall_thickness=0,
                                   img_border_also_collide=False,
                                   reflexion_order=3,
                                   should_scale_rays=True,
                                   should_scale_img=False)

# show rays on input
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=input_, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None,
                                    show_only_reflections=True)
ips.img.imshow(ray_img, size=4)

# show rays on output
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=output_, ray_value=1.3, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None,
                                    show_only_reflections=True)
ips.img.imshow(ray_img, size=4)

In [ ]:
ips.img.imshow(output_, size=5)

# calc rays
rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
                                   img_src=input_src, 
                                   directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=5),
                                   wall_values=None, 
                                   wall_thickness=1,
                                   img_border_also_collide=False,
                                   reflexion_order=3,
                                   should_scale_rays=True,
                                   should_scale_img=False)

# show rays on input
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=input_, ray_value=2, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None,
                                    show_only_reflections=True)
ips.img.imshow(ray_img, size=4)

# show rays on output
ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
                                    output_format="single_image", 
                                    img_background=output_, ray_value=1.3, ray_thickness=1, 
                                    img_shape=(256, 256), dtype=float, standard_value=0,
                                    should_scale_rays_to_image=True, original_max_width=None, original_max_height=None,
                                    show_only_reflections=True)
ips.img.imshow(ray_img, size=4)

In [ ]:
# # get image
# ips.img.imshow(output_, size=5)

# # calc rays
# rays = ips.ray_tracing.trace_beams(rel_position=[0.5, 0.5], 
#                                    img_src=input_src, 
#                                    directions_in_degree=ips.ray_tracing.get_linear_degree_range(start=0, stop=360, step_size=0.1),
#                                    wall_values=None, 
#                                    wall_thickness=0,
#                                    img_border_also_collide=False,
#                                    reflexion_order=3,
#                                    should_scale_rays=True,
#                                    should_scale_img=False)

# # show rays on input
# ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
#                                     output_format="single_image", 
#                                     img_background=input_, ray_value=2, ray_thickness=1, 
#                                     img_shape=(256, 256), dtype=float, standard_value=0,
#                                     should_scale_rays_to_image=True, original_max_width=None, original_max_height=None,
#                                     show_only_reflections=True)
# ips.img.imshow(ray_img, size=4)

# # show rays on output
# ray_img = ips.ray_tracing.draw_rays(rays, detail_draw=False, 
#                                     output_format="single_image", 
#                                     img_background=output_, ray_value=1.3, ray_thickness=1, 
#                                     img_shape=(256, 256), dtype=float, standard_value=0,
#                                     should_scale_rays_to_image=True, original_max_width=None, original_max_height=None,
#                                     show_only_reflections=True)
# ips.img.imshow(ray_img, size=4)

---